In [5]:
import pandas as pd

# Read semicolon-separated CSV, Norwegian locale for numbers
#df = pd.read_csv('12891_Etternavn.csv', sep=';', decimal=',', thousands=' ')
df = pd.read_csv('datasetsFromWeb/12891_Etternavn.csv', sep=';', decimal=',', thousands=' ')

df['Personer'] = df['Personer'].astype(str).str.replace(' ', '')
df['Personer'] = pd.to_numeric(df['Personer'].replace('.', pd.NA), errors='coerce')

#
top_10 = df.dropna().sort_values('Personer', ascending=False).head(4)

print("Top 10 most common surnames:")
#print(top_10)

#top_10.to_csv('topSurnames.csv', index=False)


Top 10 most common surnames:
             Navn  Personer
1134       Hansen   47879.0
1534     Johansen   45604.0
2388        Olsen   44682.0
1805       Larsen   34833.0
128      Andersen   34382.0
2454     Pedersen   32685.0
2263       Nilsen   31657.0
1713  Kristiansen   21809.0
1520       Jensen   21253.0
1582      Karlsen   19636.0


In [22]:
import kagglehub

#Download latest version
#path = kagglehub.dataset_download("erpel1/forenames-and-surnames-with-gender-and-country")

#print("Path to dataset files:", path)

In [23]:
#path = kagglehub.dataset_download("beridzeg45/most-common-surnames-in-the-world")

#print("Path to dataset files:", path)

In [24]:
import pandas as pd

# Read semicolon-separated CSV, Norwegian locale for numbers
#df = pd.read_csv('12891_Etternavn.csv', sep=';', decimal=',', thousands=' ')
df = pd.read_csv('FirstNames/forenames.csv', sep=',', decimal='.', thousands=' ')

df['count'] = df['count'].astype(str).str.replace(' ', '')
df['count'] = pd.to_numeric(df['count'].replace('.', pd.NA), errors='coerce')

#
top_10 = df.dropna().sort_values('count', ascending=False).head(4)

print("Top 10 most common surnames:")
print(top_10)


C:\Users\hanur\AppData\Local\Temp\ipykernel_29412\3583536464.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('FirstNames/forenames.csv', sep=',', decimal='.', thousands=' ')


Top 10 most common surnames:
        forename gender country    count
2513920     محمد      M      EG  1584172
2513921    Ahmed      M      EG  1487556
2513922  Mohamed      M      EG  1446833
2513923     احمد      M      EG   936642


In [20]:
import csv
from collections import defaultdict

# Method 1: Using csv module
def count_names_csv_method(filename):
    names_to_count = {'mohammed', 'mohamed'}
    counts = defaultdict(int)

    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            forename = row['forename'].lower().strip()
            if forename in names_to_count:
                count = int(row['count'])
                counts[forename] += count

    return counts

# Method 2: Using pandas (if you have it installed)
def count_names_pandas(filename):
    try:
        import pandas as pd
        df = pd.read_csv(filename)
        names_to_count = ['mohammed', 'mohamed']

        # Filter and sum counts
        mask = df['forename'].str.lower().isin(names_to_count)
        result = df[mask].groupby(df['forename'].str.lower())['count'].sum()

        return result.to_dict()
    except ImportError:
        print("Pandas not installed, using csv method instead")
        return count_names_csv_method(filename)

# Method 3: Simple manual parsing (no csv module needed)
def count_names_simple(filename):
    names_to_count = {'mohammed', 'mohamed'}
    counts = defaultdict(int)

    with open(filename, 'r', encoding='utf-8') as file:
        # Skip header
        next(file)

        for line in file:
            parts = line.strip().split(',')
            if len(parts) >= 4:  # Make sure we have enough columns
                forename = parts[0].lower().strip()
                if forename in names_to_count:
                    count = int(parts[3])  # count is the 4th column
                    counts[forename] += count

    return counts

# Run the code
filename = 'FirstNames/forenames.csv'  # Replace with your actual filename

# Choose one method
result = count_names_csv_method(filename)
# result = count_names_pandas(filename)  # Alternative
# result = count_names_simple(filename)  # Alternative

# Print results
print("Total counts:")
for name, count in result.items():
    print(f"{name}: {count}")

print(f"\nTotal combined: {sum(result.values())}")

Total counts:
mohammed: 924171
mohamed: 2835189

Total combined: 3759360


In [28]:
import csv
from collections import defaultdict
import operator

def analyze_names_by_country(filename, country_code='NO'):
    """
    Analyze name popularity for a specific country code
    """
    # Separate by gender
    male_names = []
    female_names = []
    all_names = []

    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['country'] == country_code:
                name = row['forename']
                gender = row['gender']
                count = int(row['count'])

                all_names.append((name, gender, count))

                if gender == 'M':
                    male_names.append((name, count))
                elif gender == 'F':
                    female_names.append((name, count))

    # Sort by count (most popular first)
    male_names.sort(key=lambda x: x[1], reverse=True)
    female_names.sort(key=lambda x: x[1], reverse=True)
    all_names.sort(key=lambda x: x[2], reverse=True)

    return {
        'all': all_names,
        'male': male_names,
        'female': female_names,
        'total_names': len(all_names),
        'total_males': len(male_names),
        'total_females': len(female_names),
        'country': country_code
    }

def print_top_names(results, top_n=10):
    """
    Print top N names for each category
    """
    print(f"\n{'='*50}")
    print(f"TOP NAMES FOR COUNTRY: {results['country']}")
    print(f"{'='*50}")

    print(f"\n📊 OVERALL TOP {top_n} NAMES:")
    print("-" * 40)
    for i, (name, gender, count) in enumerate(results['all'][:top_n], 1):
        gender_symbol = 'M' if gender == 'M' else 'W'
        print(f"{i:2}. {gender_symbol} {name:<15} {count:>8,}")

    print(f"\nM MALE TOP {top_n} NAMES:")
    print("-" * 40)
    for i, (name, count) in enumerate(results['male'][:top_n], 1):
        print(f"{i:2}. {name:<15} {count:>8,}")

    print(f"\nW FEMALE TOP {top_n} NAMES:")
    print("-" * 40)
    for i, (name, count) in enumerate(results['female'][:top_n], 1):
        print(f"{i:2}. {name:<15} {count:>8,}")

    print(f"\n📈 Summary for {results['country']}:")
    print(f"   Total unique names: {results['total_names']:,}")
    print(f"   Male names: {results['total_males']:,}")
    print(f"   Female names: {results['total_females']:,}")

# Alternative: Using pandas (more concise)
def analyze_with_pandas(filename, country_code='NO'):
    """
    Using pandas for more advanced analysis
    """
    try:
        import pandas as pd

        # Read CSV
        df = pd.read_csv(filename)

        # Filter for Norway
        norway_df = df[df['country'] == country_code].copy()

        if norway_df.empty:
            print(f"No data found for country code: {country_code}")
            return

        norway_df = norway_df.sort_values('count', ascending=False)

        # Top overall
        print(f"\n{'='*50}")
        print(f"TOP NAMES FOR COUNTRY: {country_code}")
        print(f"{'='*50}")

        print(f"\n📊 TOP 10 OVERALL:")
        print("-" * 40)
        for idx, row in norway_df.head(10).iterrows():
            gender_symbol = 'M' if row['gender'] == 'M' else 'W'
            print(f"{gender_symbol} {row['forename']:<15} {row['count']:>8,}")

        # Top by gender
        print(f"\nM TOP 10 MALE:")
        print("-" * 40)
        male_df = norway_df[norway_df['gender'] == 'M'].head(10)
        for idx, row in male_df.iterrows():
            print(f"{row['forename']:<15} {row['count']:>8,}")

        print(f"\nW TOP 10 FEMALE:")
        print("-" * 40)
        female_df = norway_df[norway_df['gender'] == 'F'].head(10)
        for idx, row in female_df.iterrows():
            print(f"{row['forename']:<15} {row['count']:>8,}")

        # Additional statistics
        print(f"\n📈 Statistics for {country_code}:")
        print(f"   Total unique names: {len(norway_df):,}")
        print(f"   Male names: {len(male_df):,}")
        print(f"   Female names: {len(female_df):,}")
        print(f"   Most common name: {norway_df.iloc[0]['forename']} ({norway_df.iloc[0]['count']:,})")

        return norway_df

    except ImportError:
        print("Pandas not installed. Install with: pip install pandas")
        return None

# Run the analysis
filename = 'FirstNames/forenames.csv'  # Replace with your actual filename

# Method 1: Using the custom function
print("Method 1: Custom CSV parsing")
results = analyze_names_by_country(filename, 'AE')
print_top_names(results, top_n=10)

def save_results_to_file(results, filename='norway_names.txt'):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"NAME ANALYSIS FOR COUNTRY: {results['country']}\n")
        f.write("="*50 + "\n\n")

        f.write("ALL NAMES (sorted by popularity):\n")
        f.write("-"*40 + "\n")
        for name, gender, count in results['all']:
            f.write(f"{name:<15} ({gender}) {count:>8,}\n")

    print(f"\n💾 Results saved to {filename}")

# Uncomment to save results
# save_results_to_file(results)

Method 1: Custom CSV parsing

TOP NAMES FOR COUNTRY: AE

📊 OVERALL TOP 10 NAMES:
----------------------------------------
 1. M Md               132,181
 2. M Muhammad          84,884
 3. M Mohammed          51,575
 4. M Abdul             48,408
 5. M Ali               40,866
 6. M Mohammad          39,532
 7. M Ahmed             31,549
 8. M Mohamed           30,575
 9. M محمد              25,879
10. M Ahmad             15,871

M MALE TOP 10 NAMES:
----------------------------------------
 1. Md               132,181
 2. Muhammad          84,884
 3. Mohammed          51,575
 4. Abdul             48,408
 5. Ali               40,866
 6. Mohammad          39,532
 7. Ahmed             31,549
 8. Mohamed           30,575
 9. محمد              25,879
10. Ahmad             15,871

W FEMALE TOP 10 NAMES:
----------------------------------------
 1. Md                 8,381
 2. Sara               6,774
 3. Fatima             5,862
 4. Maryam             3,846
 5. Mariam             3,468
 6. A

In [7]:
import pandas as pd

def top_names_by_country(input_file, output_file, countries, encoding='utf-8'):
    """
    Extract top 10 names per selected country from a CSV file.

    Parameters:
        input_file (str): Path to input CSV (no header, columns: name, gender, country, count)
        output_file (str): Path to output CSV (will contain same columns with top 10 per country)
        countries (list): List of country codes to process (e.g., ['AE', 'US'])
        encoding (str): File encoding (default 'utf-8')
    """
    columns = ['name', 'gender', 'country', 'count']
    df = pd.read_csv(input_file, names=columns, encoding=encoding)

    df['count'] = pd.to_numeric(df['count'], errors='coerce')
    df_filtered = df[df['country'].isin(countries)]

    if df_filtered.empty:
        print("No data found for the specified countries.")
        return

    top_dfs = []
    for country in countries:
        country_df = df_filtered[df_filtered['country'] == country]
        top10 = country_df.sort_values('count', ascending=False).head(10)
        top_dfs.append(top10)


    result = pd.concat(top_dfs, ignore_index=True)

    result.to_csv(output_file, index=False, encoding=encoding)
    print(f"Top 10 names per country saved to {output_file}")

if __name__ == "__main__":
    INPUT_FILE = "surenames/surnames.csv"
    OUTPUT_FILE = "top10PerCountry/top10SurenamesPerCountry.csv"
    #COUNTRIES = ['AE', 'SA', 'US']
    df = pd.read_csv(INPUT_FILE)
    uniqueCountries = df['country'].unique()
    print(uniqueCountries)

    top_names_by_country(INPUT_FILE, OUTPUT_FILE, uniqueCountries)

C:\Users\hanur\AppData\Local\Temp\ipykernel_16112\1777525783.py:39: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_FILE)


['AE' 'AF' 'AL' 'AO' 'AR' 'AT' 'AZ' 'BD' 'BE' 'BF' 'BG' 'BH' 'BI' 'BN'
 'BO' 'BR' 'BW' 'CA' 'CH' 'CL' 'CM' 'CN' 'CO' 'CR' 'CY' 'CZ' 'DE' 'DJ'
 'DK' 'DZ' 'EC' 'EE' 'EG' 'ES' 'ET' 'FI' 'FJ' 'FR' 'GB' 'GE' 'GH' 'GR'
 'GT' 'HK' 'HN' 'HR' 'HT' 'HU' 'ID' 'IE' 'IL' 'IN' 'IQ' 'IR' 'IS' 'IT'
 'JM' 'JO' 'JP' 'KH' 'KR' 'KW' 'KZ' 'LB' 'LT' 'LU' 'LY' 'MA' 'MD' 'MO'
 'MT' 'MU' 'MV' 'MX' 'MY' nan 'NG' 'NL' 'NO' 'OM' 'PA' 'PE' 'PH' 'PL' 'PR'
 'PS' 'PT' 'QA' 'RS' 'RU' 'SA' 'SD' 'SE' 'SG' 'SI' 'SV' 'SY' 'TM' 'TN'
 'TR' 'TW' 'US' 'UY' 'YE' 'ZA']


C:\Users\hanur\AppData\Local\Temp\ipykernel_16112\1777525783.py:14: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, names=columns, encoding=encoding)


Top 10 names per country saved to top10PerCountry/topSurenamesPerCountry.csv


In [11]:
import pandas as pd

def combine_names_and_surnames(
    first_names_file,
    surnames_file,
    output_file,
    top_n_first=None,
    top_m_surnames=None,
    sample_size=None,
    encoding='utf-8'
):
    """
    Combine first names and surnames by country and gender.

    Parameters:
        first_names_file (str): path to CSV with first names (name,gender,country,count)
        surnames_file (str): path to CSV with surnames (surname,gender,country,count)
        output_file (str): where to save the combined results
        top_n_first (int, optional): keep only top N first names per (country,gender) by count
        top_m_surnames (int, optional): keep only top M surnames per (country,gender) by count
        sample_size (int, optional): randomly sample this many combinations per (country,gender)
        encoding (str): file encoding (default utf-8)
    """
    # Load data
    first_df = pd.read_csv(first_names_file, encoding=encoding)
    surnames_df = pd.read_csv(surnames_file, encoding=encoding)

    # Ensure count is numeric
    first_df['count'] = pd.to_numeric(first_df['count'], errors='coerce')
    surnames_df['count'] = pd.to_numeric(surnames_df['count'], errors='coerce')

    # Drop rows with missing gender/country
    first_df = first_df.dropna(subset=['gender', 'country'])
    surnames_df = surnames_df.dropna(subset=['gender', 'country'])

    # Keep only top N first names per (country,gender)
    if top_n_first:
        first_df = (first_df
                    .sort_values(['country', 'gender', 'count'], ascending=[True, True, False])
                    .groupby(['country', 'gender'])
                    .head(top_n_first)
                    .reset_index(drop=True))

    if top_m_surnames:
        surnames_df = (surnames_df
                       .sort_values(['country', 'gender', 'count'], ascending=[True, True, False])
                       .groupby(['country', 'gender'])
                       .head(top_m_surnames)
                       .reset_index(drop=True))

    first_df = first_df.rename(columns={'name': 'first_name', 'count': 'first_count'})
    surnames_df = surnames_df.rename(columns={'name': 'last_name', 'count': 'last_count'})

    combined = pd.merge(first_df, surnames_df, on=['country', 'gender'], how='inner')

    combined['combined_count'] = combined['first_count'] * combined['last_count']

    result = combined[['first_name', 'last_name', 'gender', 'country', 'combined_count']]

    # Optional: sample a fixed number of rows per group
    #Other solution is if surename gender = null
    #I think that this means that the surenamesname applies to both genders
    #
    if sample_size:
        result = (result
                  .groupby(['country', 'gender'])
                  .apply(lambda x: x.sample(n=min(sample_size, len(x)), random_state=42),
                         include_groups=False)   #
                  .reset_index())

    result.to_csv(output_file, index=False, encoding=encoding)
    print(f"Combined data saved to {output_file} with {len(result)} rows.")

# -------------------------------------------------------------------
# Example usage:
if __name__ == "__main__":
    combine_names_and_surnames(
        first_names_file="top10PerCountry/top10ForenamesPerCountry.csv",
        surnames_file="top10PerCountry/top10SurenamesPerCountry.csv",
        output_file="full_names.csv",
        top_n_first=50,
        top_m_surnames=50,
        sample_size=1000
    )

Combined data saved to full_names.csv with 5873 rows.


In [ ]:
import pandas as pd

def combine_names_and_surnames(
    first_names_file,
    surnames_file,
    output_file,
    top_n_first=None,
    top_m_surnames=None,
    sample_size=None,
    encoding='utf-8'
):
    """
    Combine first names and surnames by country and gender.
    Handles surnames with null gender by treating them as gender‑neutral
    and expanding them to both 'M' and 'F'.
    """
    # --- Load data ---
    first_df = pd.read_csv(first_names_file, encoding=encoding)
    surnames_df = pd.read_csv(surnames_file, encoding=encoding)

    # Ensure count is numeric
    first_df['count'] = pd.to_numeric(first_df['count'], errors='coerce')
    surnames_df['count'] = pd.to_numeric(surnames_df['count'], errors='coerce')

    # Drop rows with missing country (essential for grouping)
    first_df = first_df.dropna(subset=['country'])
    surnames_df = surnames_df.dropna(subset=['country'])

    # For first names, drop rows with missing gender (assume they are all specified)
    first_df = first_df.dropna(subset=['gender'])

    # --- Handle gender‑neutral surnames ---
    # Split surnames into those with gender and those without
    surnames_with_gender = surnames_df[surnames_df['gender'].notna()].copy()
    surnames_neutral = surnames_df[surnames_df['gender'].isna()].copy()

    # Expand neutral surnames to both genders (preserving the original count)
    if not surnames_neutral.empty:
        neutral_m = surnames_neutral.assign(gender='M')
        neutral_f = surnames_neutral.assign(gender='F')
        surnames_df = pd.concat([surnames_with_gender, neutral_m, neutral_f], ignore_index=True)
    else:
        surnames_df = surnames_with_gender

    # Now surnames_df has no null gender; any remaining nulls (if any) can be dropped
    surnames_df = surnames_df.dropna(subset=['gender'])

    # --- Keep only top N first names per (country, gender) ---
    if top_n_first:
        first_df = (first_df
                    .sort_values(['country', 'gender', 'count'], ascending=[True, True, False])
                    .groupby(['country', 'gender'])
                    .head(top_n_first)
                    .reset_index(drop=True))

    # --- Keep only top M surnames per (country, gender) ---
    if top_m_surnames:
        surnames_df = (surnames_df
                       .sort_values(['country', 'gender', 'count'], ascending=[True, True, False])
                       .groupby(['country', 'gender'])
                       .head(top_m_surnames)
                       .reset_index(drop=True))

    # --- Prepare for merge ---
    first_df = first_df.rename(columns={'name': 'first_name', 'count': 'first_count'})
    surnames_df = surnames_df.rename(columns={'name': 'last_name', 'count': 'last_count'})

    # Inner join on country and gender – only groups present in both files will appear
    combined = pd.merge(first_df, surnames_df, on=['country', 'gender'], how='inner')

    # Compute combined count (product) as a popularity indicator
    combined['combined_count'] = combined['first_count'] * combined['last_count']

    # Keep relevant columns
    result = combined[['first_name', 'last_name', 'gender', 'country', 'combined_count']]

    # --- Optional: sample a fixed number of rows per (country, gender) ---
    if sample_size:
        result = (result
                  .groupby(['country', 'gender'])
                  .apply(lambda x: x.sample(n=min(sample_size, len(x)), random_state=42),
                         include_groups=False)
                  .reset_index())  # keeps country and gender as columns

    # Save to CSV
    result.to_csv(output_file, index=False, encoding=encoding)
    print(f"Combined data saved to {output_file} with {len(result)} rows.")

# -------------------------------------------------------------------
# Example usage:
if __name__ == "__main__":
    combine_names_and_surnames(
        first_names_file="top10PerCountry/top10ForenamesPerCountry.csv",
        surnames_file="top10PerCountry/top10SurenamesPerCountry.csv",
        output_file="full_names.csv",
        top_n_first=50,
        top_m_surnames=50,
        sample_size=1000
    )

In [ ]:
import pandas as pd

def combine_names_and_surnames(
    first_names_file,
    surnames_file,
    output_file,
    top_n_first=None,
    top_m_surnames=None,
    sample_size=None,
    encoding='utf-8'
):
    """
    Combine first names and surnames by country and gender.
    Handles surnames with null gender by treating them as gender‑neutral
    and expanding them to both 'M' and 'F'.
    """
    # --- Load data ---
    first_df = pd.read_csv(first_names_file, encoding=encoding)
    surnames_df = pd.read_csv(surnames_file, encoding=encoding)

    # Ensure count is numeric
    first_df['count'] = pd.to_numeric(first_df['count'], errors='coerce')
    surnames_df['count'] = pd.to_numeric(surnames_df['count'], errors='coerce')

    # Drop rows with missing country (essential for grouping)
    first_df = first_df.dropna(subset=['country'])
    surnames_df = surnames_df.dropna(subset=['country'])

    # For first names, drop rows with missing gender (assume they are all specified)
    first_df = first_df.dropna(subset=['gender'])

    # --- Handle gender‑neutral surnames ---
    # Split surnames into those with gender and those without
    surnames_with_gender = surnames_df[surnames_df['gender'].notna()].copy()
    surnames_neutral = surnames_df[surnames_df['gender'].isna()].copy()

    # Expand neutral surnames to both genders (preserving the original count)
    if not surnames_neutral.empty:
        neutral_m = surnames_neutral.assign(gender='M')
        neutral_f = surnames_neutral.assign(gender='F')
        surnames_df = pd.concat([surnames_with_gender, neutral_m, neutral_f], ignore_index=True)
    else:
        surnames_df = surnames_with_gender

    # Now surnames_df has no null gender; any remaining nulls (if any) can be dropped
    surnames_df = surnames_df.dropna(subset=['gender'])

    # --- Keep only top N first names per (country, gender) ---
    if top_n_first:
        first_df = (first_df
                    .sort_values(['country', 'gender', 'count'], ascending=[True, True, False])
                    .groupby(['country', 'gender'])
                    .head(top_n_first)
                    .reset_index(drop=True))

    # --- Keep only top M surnames per (country, gender) ---
    if top_m_surnames:
        surnames_df = (surnames_df
                       .sort_values(['country', 'gender', 'count'], ascending=[True, True, False])
                       .groupby(['country', 'gender'])
                       .head(top_m_surnames)
                       .reset_index(drop=True))

    # --- Prepare for merge ---
    first_df = first_df.rename(columns={'name': 'first_name', 'count': 'first_count'})
    surnames_df = surnames_df.rename(columns={'name': 'last_name', 'count': 'last_count'})

    # Inner join on country and gender – only groups present in both files will appear
    combined = pd.merge(first_df, surnames_df, on=['country', 'gender'], how='inner')

    # Compute combined count (product) as a popularity indicator
    combined['combined_count'] = combined['first_count'] * combined['last_count']

    # Keep relevant columns
    result = combined[['first_name', 'last_name', 'gender', 'country', 'combined_count']]

    # --- Optional: sample a fixed number of rows per (country, gender) ---
    if sample_size:
        result = (result
                  .groupby(['country', 'gender'])
                  .apply(lambda x: x.sample(n=min(sample_size, len(x)), random_state=42),
                         include_groups=False)
                  .reset_index())  # keeps country and gender as columns

    # Save to CSV
    result.to_csv(output_file, index=False, encoding=encoding)
    print(f"Combined data saved to {output_file} with {len(result)} rows.")

# -------------------------------------------------------------------
# Example usage:
if __name__ == "__main__":
    combine_names_and_surnames(
        first_names_file="top10PerCountry/top10ForenamesPerCountry.csv",
        surnames_file="top10PerCountry/top10SurenamesPerCountry.csv",
        output_file="full_names.csv",
        top_n_first=50,
        top_m_surnames=50,
        sample_size=1000
    )

In [ ]:
import pandas as pd
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="<YOUR_OPENROUTER_API_KEY>",
)
#api_key="<YOUR_OPENROUTER_API_KEY>",

completion = client.chat.completions.create(
  model="anthropic/claude-3-haiku",
  messages=[{"role": "user", "content": "Hello"}]
)
print(completion.choices[0].message.content)
